## Этап 1. Импортирование библиотек  и данных. Первичное знакомство с данными

### 1.1  Импортирование библиотек

Загрузим необходимые библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option("display.max_colwidth", -1)

### 1.2 Подключение к базе данных

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'xxx', # пароль
 'host': 'xxx',
 'port': xxx, # порт подключения
 'db': 'xxx'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Создадим функцию для выполнения SQL запросов:

In [3]:
def run_sql(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

### 1.3 Первичное знакомство с данными

Для начала рассмотрим наши данные, создав запросы и выполнив SQL запросы, начнем с книг:

In [4]:
books = """
SELECT 
    *
FROM 
   books;
"""
run_sql(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


Таблица содержит 1000 строк и 6 столбцов, содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Посмотрим общую информацию:

In [5]:
run_sql(books).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


Пропусков нет. Единственном отметим, что дата публикация имеет не временной тип данных. 

In [6]:
run_sql(books).duplicated().sum()

0

Явных дубликатов нет, проверим также чуть иначе: 

In [7]:
run_sql(books)['title'].duplicated().sum()

1

Посмотрим данную книгу:

In [8]:
run_sql(books)[run_sql(books)['title'].duplicated()]

,book_id,author_id,title,num_pages,publication_date,publisher_id
426,427,39,Memoirs of a Geisha,503,2005-11-22,311


Найдем соответствующее название:

In [9]:
run_sql(books).query('title == "Memoirs of a Geisha"')

,book_id,author_id,title,num_pages,publication_date,publisher_id
425,426,39,Memoirs of a Geisha,434,2005-11-15,241
426,427,39,Memoirs of a Geisha,503,2005-11-22,311


Как минимум книга от другого издадельства, все в порядке.


Познакомимся со следующей таблицей, с авторами:

In [10]:
authors= """
SELECT 
    *
FROM 
    authors;
"""
run_sql(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


Таблица содержит 636 строк и 2 столбца, содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Посмотрим общую информацию:

In [11]:
run_sql(authors).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


Пропусков нет, типы данных соответствующие. 

In [12]:
run_sql(authors).duplicated().sum()

0

Дубликаты отсуствуют.

Следующая таблица, с издетельствами:

In [13]:
publishers = """
SELECT 
     *
FROM 
    publishers;
"""
run_sql(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


Таблица содержит 340 строк, 2 столбца, содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Посзнакомимся с общей информацией:

In [14]:
run_sql(publishers).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


Пропусков нет, типы соответствующие. Проверим также на дубликаты:

In [15]:
run_sql(publishers).duplicated().sum()

0

Дубликатов нет. 

Перейдем к таблице с рейтингами книг:

In [16]:
ratings = """
SELECT 
    *
FROM 
    ratings;
"""
run_sql(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


Таблица содержит 6456 строк, 4 столбца, содержит данные о пользовательских обзорах:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Познакомимся поближе:

In [17]:
run_sql(ratings).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


Пропусков нет, типы соответствующие. Проверим на дубликаты:

In [18]:
run_sql(ratings).duplicated().sum()

0

Явных дубликатов нет.

Осталась последняя таблица, данные о пользовательских обзорах:

In [19]:
reviews = """
SELECT 
    *
FROM 
    reviews;
"""
run_sql(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same enter true practice student system.
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy population. Pm vote take. Long born close thought person old. Visit ever school manage about.
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone foreign amount within cell. Ball poor hot that kid.


Таблица содержит 2793 строки, 4 столбца, содержит данные о пользовательских обзорах:
review_id — идентификатор обзора;
book_id — идентификатор книги;
username — имя автора обзора;
text — текст обзора.

Проверим подробнее:

In [20]:
run_sql(reviews).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Пропусков нет, типы данных соотетствующие. Проверим также на дубликаты:

In [21]:
run_sql(reviews).duplicated().sum()

0

Явных дубликтов нет. 

## Этап 2. Поставленные задания

### 2.1 Cколько книг вышло после 1 января 2000 года?

Расчитаем количество книг, выщедших после 1 января 2000 года:

In [22]:
count_books_after_2000 = """
SELECT 
    COUNT (*) AS count_books_after_2000
FROM 
    books
WHERE
    publication_date >= '2000-01-01';
"""
                
run_sql(count_books_after_2000)

,count_books_after_2000
0,821


Такм образом с 1 января 2000 года, в базе хранится информация о 821-ой книге.

### 2.2 Для каждой книги посчитать количество обзоров и среднюю оценку

Расчитаем количество обзоров и среднюю оценку для каждой книги, за исключением тех, где нет обзоров(6 книг) :

In [23]:
count_reviews_and_avg_rating = """
SELECT 
    books.title,
    sub_cnt.cnt_reviews,
    sub_avg.avg_rating
        
FROM 
    books LEFT JOIN (
    SELECT 
        book_id, 
        AVG(rating) AS avg_rating
    FROM 
        ratings
    GROUP BY
        book_id) AS sub_avg ON sub_avg.book_id = books.book_id
    LEFT JOIN
    (SELECT
        book_id,
        COUNT(review_id) AS cnt_reviews
     FROM
         reviews
     GROUP BY
         book_id) AS sub_cnt on sub_cnt.book_id = books.book_id
WHERE
cnt_reviews is not null
ORDER bY
    cnt_reviews DESC, avg_rating DESC; 
""" 
                
run_sql(count_reviews_and_avg_rating)

,title,cnt_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
989,Death: The High Cost of Living,1,3.000000
990,Moo Baa La La La!,1,3.000000
991,1 000 Places to See Before You Die,1,2.500000
992,History of Beauty,1,2.500000


994 строки, имеющие обзоры и рейтинг. Строки расталвены в порядке убывания количества обзоров и среднего рейтинга. Больше всего обзоров на Twilight (Сумерки), здесь 7 обзоров со средним рейтингом 3.66. Следом идет Harry Potter and the Prisoner of Azkaban (Гарри Поттер и узник Азкабана) с 6 рейтингами и средним рейтингом 4.41. Замыкает тройку  лидеров Harry Potter and the Chamber of Secrets (Гарри Поттер и Тайная комната) со средним рейтингом 4.29. 

In [24]:
count_reviews_and_avg_rating = """
SELECT 
    books.title,
    sub_cnt.cnt_reviews,
    sub_avg.avg_rating
        
FROM 
    books LEFT JOIN (
    SELECT 
        book_id, 
        AVG(rating) AS avg_rating
    FROM 
        ratings
    GROUP BY
        book_id) AS sub_avg ON sub_avg.book_id = books.book_id
    LEFT JOIN
    (SELECT
        book_id,
        COUNT(review_id) AS cnt_reviews
     FROM
         reviews
     GROUP BY
         book_id
          ) AS sub_cnt on sub_cnt.book_id = books.book_id

ORDER bY
    cnt_reviews  DESC, avg_rating DESC; 
""" 
                
run_sql(count_reviews_and_avg_rating)

,title,cnt_reviews,avg_rating
0,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.000000
1,Disney's Beauty and the Beast (A Little Golden Book),NaN,4.000000
2,Leonardo's Notebooks,NaN,4.000000
3,Essential Tales and Poems,NaN,4.000000
4,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.666667
...,...,...,...
995,Moo Baa La La La!,1.0,3.000000
996,Death: The High Cost of Living,1.0,3.000000
997,History of Beauty,1.0,2.500000
998,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,1.0,2.500000


### 2.3 Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключим из анализа брошюры;

Поссмотрим на топ-5 издадельств, выпустивших книги толще 50 страниц:

In [25]:
top_publishers = '''
SELECT
    publisher,
    COUNT(Sub_book.book_id) AS books_cnt
FROM
    publishers  LEFT JOIN 
    (SELECT *
     FROM
        books 
     WHERE
        num_pages > 50) AS Sub_book
        ON Sub_book.publisher_id = publishers.publisher_id
GROUP BY
    publisher
ORDER BY
    books_cnt DESC
LIMIT 5;
'''
run_sql(top_publishers)

,publisher,books_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Первое место у Penguin Books  - 42 книги. Следом идет издательство Vintage - 31 книга. Замыкает тройку -  Grand Central Publishing с количеством 25 книг. 4 место у Penguin Classics - 24 книги и замыкает топ-5 издательство Bantam	с 19 книгами.

### 2.4 Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;

Найдем топ-5 авторов с самой высокой оценкой книг:

In [26]:
top_autors = """
SELECT 
    aut_avg.author,
    AVG(aut_avg.avg_rating) AS avg_avg_rating
FROM
    (SELECT 
        authors.author AS author,
        AVG(ratings.rating) AS avg_rating
    FROM 
        books LEFT JOIN 
        authors ON books.author_id = authors.author_id
            LEFT JOIN 
        ratings ON books.book_id = ratings.book_id
    GROUP BY 
        books.book_id,
        authors.author_id
    HAVING
        COUNT(ratings.rating_id) >=50 ) AS aut_avg 
GROUP BY    
    aut_avg.author
ORDER BY
    avg_avg_rating DESC
LIMIT 5
    
; """
run_sql(top_autors)

,author,avg_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Автор с самой высокой оценкой - J.K. Rowling/Mary GrandPré со средней оценкой 4.28. На втором месте - Markus Zusak/Cao Xuân Việt Khương со средней оценкой 4.26, замыкает тройку J.R.R. Tolkien	 - 4.26. 4 место у Louisa May Alcott со средней оценкой 4.19, замыкает топ-5 Rick Riordan со средней оценкой 4.08.

### 2.5 Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Посмотрим среднее количество обзоров от активных пользователей:

In [27]:
mean__count_reviews = '''
SELECT
    AVG(sub.count)
FROM
    (SELECT 
        COUNT(*)
    FROM 
        reviews
    WHERE 
        username IN
    (SELECT 
        username
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating) > 50)
    GROUP BY 
        username)
    AS sub;
'''
run_sql(mean__count_reviews)        

,avg
0,24.333333


Таким образом среднее количество обзоров от активных пользователей - 24 обзора. 

## Этап 3. Общий вывод

В нашем распоряжении было 5 таблиц:
* books - содержит 1000 строк и 6 столбцов, содержит данные о книгах;
* authors - содержит 636 строк и 2 столбца, содержит данные об авторах;
* publishers - содержит 340 строк, 2 столбца, содержит данные об издательствах;
* rating - содержит 6456 строк, 4 столбца, содержит данные о пользовательских обзорах;
* reviews - содержит 2793 строки, 4 столбца, содержит данные о пользовательских обзорах.

В ходе работы были даны ответы на 5 поставленных вопрос:
* Cколько книг вышло после 1 января 2000 года? с 1 января 2000 года, в базе хранится информация о 821-ой книге.
* Для каждой книги посчитать количество обзоров и среднюю оценку. Расчет был произведен, топ-3 книги :
    * Twilight (Сумерки) с количеством	7 отзывом и с со средним рейтингом - 3.662500;
    * Harry Potter and the Prisoner of Azkaban (Гарри Поттер и узник Азкабана) с количеством обзоров -6 и со средним рейтингом - 4.414634;
    * Harry Potter and the Chamber of Secrets (Гарри Поттер и тайная комната) с количеством обзоров -6 и со средним рейтингом 4.287500.
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключим из анализа брошюры.Определели топ -5:
    *  Penguin Books - 42 книги;
    *  Vintage - 31 книга;
    *  Grand Central Publishing - 25 книг;
    *  Penguin Classics - 24 книги;
    *  Bantam - 19 книг.
* Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками. Определи топ-5:
    * J.K. Rowling/Mary GrandPré со средней оценкой 4.28;
    * Markus Zusak/Cao Xuân Việt Khương со средней оценкой 4.26;
    * J.R.R. Tolkien -со средней оценкой  4.26;
    * Louisa May Alcott со средней оценкой 4.19;
    * Rick Riordan со средней оценкой 4.08.
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.Таким образом среднее количество обзоров от активных пользователей. При подсчете было выявлено - 24 обзора.
    